In [16]:
# Importación de librerías
import pandas as pd  # Para manipulación y análisis de datos
from sklearn.model_selection import train_test_split  # Para dividir el conjunto de datos en entrenamiento y prueba
from sklearn.preprocessing import StandardScaler  # Para estandarizar características
from tensorflow.keras.models import Sequential  # Para inicializar modelos secuenciales
from tensorflow.keras.layers import Dense, Input  # Para definir capas densamente conectadas en la red neuronal
import joblib  # Para guardar y cargar modelos entrenados
import numpy as np  # Para cálculos numéricos
from sklearn.metrics import mean_squared_error, r2_score  # Para calcular métricas de rendimiento del modelo

In [30]:
# Cargar datos
df = pd.read_csv("juadores_fantasy.csv.csv")

#Impresion de las columnas existentes 
print(df.columns)



Index(['Nombre', 'Precio', 'Media', 'Total_puntos', 'posicion', 'equipo'], dtype='object')


In [18]:
# Dividir los datos en características (X) y el objetivo (y)
X = df[['Precio', 'Media']]
y_puntos = df['Total_puntos']
y_precio = df['Precio']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_puntos_train, y_puntos_test = train_test_split(X, y_puntos, test_size=0.2, random_state=42)
X_train, X_test, y_precio_train, y_precio_test = train_test_split(X, y_precio, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
# Crear el modelo para predecir puntos
model_puntos = Sequential([
    Input(shape=(X_train.shape[1],)),  
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

# Configuración del modelo para que utilice el algoritmo de optimización 
model_puntos.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir puntos
model_puntos.fit(X_train_scaled, y_puntos_train, epochs=50, batch_size=32, verbose=0)


In [20]:
# Crear el modelo para predecir precio
model_precio = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

# Configuración del modelo para que utilice el algoritmo de optimización 
model_precio.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir precio
model_precio.fit(X_train_scaled, y_precio_train, epochs=50, batch_size=32, verbose=0)


In [21]:
# Hacer predicciones para puntos y precio
#puntos_predictions = model_puntos.predict(X_test_scaled)
#precio_predictions = model_precio.predict(X_test_scaled)

# Obtener el jugador con más puntos y el jugador con el precio más alto
#indice_max_puntos = puntos_predictions.argmax()
#indice_max_precio = precio_predictions.argmax()

#jugador_mas_puntos = df.loc[indice_max_puntos, 'Nombre']
#jugador_mas_precio = df.loc[indice_max_precio, 'Nombre']

#print(f"Jugador con más puntos predichos: {jugador_mas_puntos}")
#print(f"Jugador con precio más alto predicho: {jugador_mas_precio}")


In [22]:
# Hacer predicciones para puntos y precio
puntos_predictions = model_puntos.predict(X_test_scaled)
precio_predictions = model_precio.predict(X_test_scaled)

# Obtener los 10 jugadores con más puntos predichos
top_10_jugadores_puntos = df.nlargest(10, 'Total_puntos')

# Obtener los 10 jugadores con los precios más altos predichos
top_10_jugadores_precio = df.nlargest(10, 'Precio')

# Impresion de los top 10 Jugadores con mayores puntos y precios 
print("Top 10 jugadores con más puntos predichos:")
print(top_10_jugadores_puntos[['Nombre', 'Total_puntos']])

print("\nTop 10 jugadores con precios más altos predichos:")
print(top_10_jugadores_precio[['Nombre', 'Precio']])



4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Top 10 jugadores con más puntos predichos:
          Nombre  Total_puntos
0  J. Bellingham           173
1        T. Kubo           147
2   A. Griezmann           144
3    I. Williams           139
4     B. Mayoral           138
5      A. García           137
6      A. Dovbyk           130
7      Á. Valles           128
8         Pepelu           127
9     A. Budimir           127

Top 10 jugadores con precios más altos predichos:
            Nombre    Precio
2     A. Griezmann  24384000
0    J. Bellingham  24262000
56  R. Lewandowski  20994000
1          T. Kubo  20904000
5        A. García  20771000
18     F. Valverde  20115000
10            Isco  19605000
13         Rodrygo  19338000
3      I. Williams  18358000
22       G. Moreno  18341000


In [23]:
# Hacer predicciones para puntos y precio
puntos_predictions = model_puntos.predict(X_test_scaled)
precio_predictions = model_precio.predict(X_test_scaled)

# Calcular el Error Cuadrático Medio (MSE) para puntos y precio
mse_puntos = mean_squared_error(y_puntos_test, puntos_predictions)
mse_precio = mean_squared_error(y_precio_test, precio_predictions)

# Calcular el coeficiente de determinación (R^2) para puntos y precio
r2_puntos = r2_score(y_puntos_test, puntos_predictions)
r2_precio = r2_score(y_precio_test, precio_predictions)

print(f"MSE para puntos: {mse_puntos:.2f}")
print(f"R^2 para puntos: {r2_puntos:.2f}")

print(f"MSE para precio: {mse_precio:.2f}")
print(f"R^2 para precio: {r2_precio:.2f}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 432us/step
MSE para puntos: 366.67
R^2 para puntos: 0.71
MSE para precio: 58074596645481.99
R^2 para precio: -0.59


In [24]:
# Crear un DataFrame con las predicciones
# Obtener los 10 jugadores con las predicciones más altas de puntos
top_10_puntos_predictions = pd.DataFrame({
    'Nombre': df.iloc[X_test.index]['Nombre'],  # Recuperar los nombres de los jugadores
    'Posicion': df.iloc[X_test.index]['posicion'],   # Utilizar la posición de los jugadores
    'Predicciones_Puntos': puntos_predictions.flatten(),  # Aplanar el array de predicciones
    'Predicciones_Precio': precio_predictions.flatten()  # Aplanar el array de predicciones
})

# Ordenar el DataFrame por Predicciones_Puntos en orden descendente
top_10_puntos_predictions = top_10_puntos_predictions.sort_values(by='Predicciones_Puntos', ascending=False)

# Tomar solo las primeras 10 filas (los 10 jugadores con las predicciones más altas)
top_10_puntos_predictions = top_10_puntos_predictions.head(10)

# Cambiar el nombre de la columna 'Predicciones_Puntos' a 'Puntos'
top_10_puntos_predictions = top_10_puntos_predictions.rename(columns={'Predicciones_Puntos': 'Puntos', 'Predicciones_Precio': 'Precio'})

# Guardar el DataFrame en un archivo CSV
top_10_puntos_predictions.to_csv('top_10_predicciones_puntos.csv', index=False)


In [25]:
top_10_puntos_predictions

,Nombre,Posicion,Puntos,Precio
0,J. Bellingham,mediocentro,194.411407,4688.642578
2,A. Griezmann,delantero,157.679214,3865.565186
399,M. Guiu,delantero,139.167618,2689.641357
10,Isco,mediocentro,134.365158,3224.895508
101,F. De Jong,mediocentro,124.485336,2951.586182
22,G. Moreno,delantero,123.424034,2947.037354
11,Sávio,delantero,122.520119,2917.972900
9,A. Budimir,delantero,118.538658,2760.377441
18,F. Valverde,mediocentro,118.007652,2857.256592
72,V. Muriqi,delantero,110.673912,2492.707764


In [26]:
# Guardar las predicciones en un archivo CSV
#predicciones_df.to_csv('predicciones.csv', index=False)

In [27]:
#joblib.dump(model_puntos, 'redes_neuronales.pkl')

In [28]:
####

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Input
from sklearn.metrics import mean_squared_error

# Cargar datos
df = pd.read_csv("juadores_fantasy.csv.csv")

# Dividir los datos en características (X) y el objetivo (y) para el modelo general
X_general = df[['Precio', 'Media']]
y_puntos_general = df['Total_puntos']
y_precio_general = df['Precio']

# Función para crear y entrenar un modelo
def entrenar_modelo(X_train, y_train):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    model = Sequential([
        Input(shape=(X_train.shape[1],)),  
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=0)
    return model, scaler

# Diccionario para almacenar los modelos
modelos_puntos = {}
modelos_precio = {}

# Lista de posiciones para modelos específicos
posiciones = ['Delantero', 'Mediocentro', 'Portero', 'Defensa']

# Entrenar un modelo para cada posición y uno general
for posicion in posiciones + ['General']:
    if posicion == 'General':
        X_train, X_test, y_train_puntos, y_test_puntos = train_test_split(X_general, y_puntos_general, test_size=0.2, random_state=42)
        X_train, X_test, y_train_precio, y_test_precio = train_test_split(X_general, y_precio_general, test_size=0.2, random_state=42)
    else:
        df_posicion = df[df['posicion'] == posicion]
        if df_posicion.shape[0] < 2:
            print(f"Not enough data for position {posicion} to train a model.")
            continue
        
        X = df_posicion[['Precio', 'Media']]
        y_puntos = df_posicion['Total_puntos']
        y_precio = df_posicion['Precio']
        X_train, X_test, y_train_puntos, y_test_puntos = train_test_split(X, y_puntos, test_size=0.2, random_state=42)
        X_train, X_test, y_train_precio, y_test_precio = train_test_split(X, y_precio, test_size=0.2, random_state=42)
    
    if X_train.shape[0] > 0:
        modelos_puntos[posicion], scaler_puntos = entrenar_modelo(X_train, y_train_puntos)
        X_test_scaled_puntos = scaler_puntos.transform(X_test)
        y_pred_puntos = modelos_puntos[posicion].predict(X_test_scaled_puntos)
        mse_puntos = mean_squared_error(y_test_puntos, y_pred_puntos)
        print(f"MSE para puntos {posicion}: {mse_puntos:.2f}")

        modelos_precio[posicion], scaler_precio = entrenar_modelo(X_train, y_train_precio)
        X_test_scaled_precio = scaler_precio.transform(X_test)
        y_pred_precio = modelos_precio[posicion].predict(X_test_scaled_precio)
        mse_precio = mean_squared_error(y_test_precio, y_pred_precio)
        print(f"MSE para precio {posicion}: {mse_precio:.2f}")
    else:
        print(f"Not enough data to train model for position {posicion}")

# Ejemplo de cómo usar el modelo específico de 'Delantero'
# Asumiendo que necesitas hacer una nueva predicción para un delantero
# Supongamos que tienes un nuevo delantero con Precio 10 y Media 7
if 'Delantero' in modelos_puntos and 'Delantero' in modelos_precio:
    nuevo_delantero = [[10, 7]]
    nuevo_delantero_scaled_puntos = scaler_puntos.transform(nuevo_delantero)
    puntos_predichos_delantero = modelos_puntos['Delantero'].predict(nuevo_delantero_scaled_puntos)
    print("Puntos predichos para el delantero:", puntos_predichos_delantero[0][0])

    nuevo_delantero_scaled_precio = scaler_precio.transform(nuevo_delantero)
    precio_predicho_delantero = modelos_precio['Delantero'].predict(nuevo_delantero_scaled_precio)
    print("Precio predicho para el delantero:", precio_predicho_delantero[0][0])
else:
    print("No hay modelo disponible para la posición 'Delantero'")


Not enough data for position Delantero to train a model.
Not enough data for position Mediocentro to train a model.
Not enough data for position Portero to train a model.
Not enough data for position Defensa to train a model.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE para puntos General: 402.48
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/stepWARNING:tensorflow:5 out of the last 17 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x30f0470d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflo